## 1. Introdução

O presente projeto tem como objetivo analisar os resultados de um experimento A/B conduzido pelo iFood para avaliar o impacto de uma campanha de cupons sobre a retenção de usuários. O iFood, uma das maiores empresas de tecnologia da América Latina, utiliza constantemente testes A/B para validar hipóteses de crescimento e viabilidade de novas funcionalidades.

Neste contexto, fui contratado como Analista de Dados para investigar os resultados da campanha, mensurar seu impacto, analisar sua viabilidade financeira e propor melhorias com base nos dados disponibilizados. Ao longo deste estudo, também será explorada a possibilidade de segmentar usuários de forma estratégica para otimizar futuras ações de marketing e retenção.


## 2. Entendimento dos Dados

A análise foi conduzida com base em quatro datasets fornecidos:

- **Pedidos** (order.json.gz): Contém cerca de 3,6 milhões de pedidos realizados entre dez/2018 e jan/2019, com informações detalhadas de transações, usuários e restaurantes envolvidos.
- **Usuários** (consumer.csv.gz): Base com aproximadamente 806 mil usuários, contendo dados cadastrais e indicadores de atividade.
- **Restaurantes** (restaurant.csv.gz): Informações de cerca de 7 mil estabelecimentos, com dados como localização, faixa de preço e ticket médio.
- **Marcação do Teste A/B** (ab_test_ref.tar.gz): Lista de usuários com a identificação do grupo ao qual pertencem no experimento (controle ou exposto à campanha).

Esses dados foram integrados e processados de forma a permitir uma análise cruzada entre comportamento de consumo, grupo experimental e atributos dos usuários e restaurantes. Antes da análise exploratória, foi realizado um processo de limpeza e preparação dos dados, seguindo boas práticas de ETL e de construção de pipelines.

## 3. Metodologia

A abordagem analítica adotada seguiu as seguintes etapas:

- **Definição de Métricas-Chave**: Foram selecionados indicadores relevantes para avaliar o sucesso da campanha de cupons, como:
- **Taxa de conversão**: proporção de usuários que realizaram pedidos.
- **Ticket Médio por Usuário**: total gasto dividido pelo número de usuários.
- **Ticket Médio por Pedido**: valor médio por pedido.
- **Frequência de pedidos**: número médio de pedidos por usuário.
- **Comparação entre Grupos**: As métricas foram calculadas separadamente para o grupo de controle e para o grupo impactado pela campanha. Foram aplicados testes estatísticos para verificar se as diferenças observadas são estatisticamente significativas.
- **Análise de Viabilidade Financeira**: Com base nos resultados, foi estimado o impacto financeiro da campanha, considerando premissas realistas para custo do cupom e receita adicional gerada.
- **Criação de Segmentações**: Usuários foram agrupados com base em comportamentos e características comuns, com o objetivo de analisar a efetividade da campanha dentro de cada segmento e propor estratégias personalizadas.
- **Recomendações Estratégicas**: Com base nos insights obtidos, foram sugeridas ações futuras para o iFood, com estimativas de impacto e viabilidade.

Para assegurar que as diferenças observadas entre o grupo de controle (GC) e o grupo exposto à campanha (PA) não ocorreram por acaso, é fundamental considerar o tamanho amostral e a margem de erro da análise.

- População total: 806.467 usuários
- Participação do grupo de controle (GC): 44,7%
- Tamanho do grupo de controle: 360.542 usuários
- Participação do grupo abordado (PA): 55,3%
- Margem de erro estimada (com população conhecida): ±0,16 ponto percentual

Essa margem de erro é extremamente baixa, o que indica que o tamanho amostral é robusto o suficiente para identificar diferenças estatisticamente significativas entre os grupos com alto nível de confiança.

Ao longo da análise, foram utilizados testes estatísticos para validar as hipóteses levantadas (como diferença nas taxas de conversão ou diferença de ticket médio entre GC e PA). Como o erro máximo é de apenas 0,16 p.p., diferenças maiores que esse valor podem ser interpretadas com segurança como estatisticamente relevantes.

## 4. Questão 1

No iFood, várias áreas utilizam testes A/B para avaliar o impacto de
ações em diferentes métricas. Esses testes permitem validar hipóteses
de crescimento e a viabilidade de novas funcionalidades em um grupo
restrito de usuários. Nos dados fornecidos nesse case você encontrará
uma marcação de usuários, separando-os entre grupo teste e controle
de uma campanha de cupons, que disponibilizou para os usuários do
grupo teste um cupom especial.

**a)** Defina os indicadores relevantes para mensurar o sucesso da
campanha e analise se ela teve impacto significativo dentro do
período avaliado. 

- **Taxa de conversão** (usuários que fizeram pedido / usuários expostos)
- **Receita média por usuário** (receita total / número de usuários)
- **Ticket médio por pedido** (receita total / número de pedidos)
- **Frequência de pedidos** (número de pedidos / número de usuários)

### 4.1 Taxa de Conversão

Verificar dos usuários que são PA / GC, quantos estão presentes na tabela orders. Comparar as taxas de conversão para entender se existe lift.

In [0]:
%sql
SELECT 
    is_target,
    COUNT(DISTINCT customer_id) AS distinct_customer_count
FROM ifood_bronze.ab_test_ref
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY is_target

is_target,distinct_customer_count
control,360542
target,445925


In [0]:
%sql
SELECT 
    is_target,
    pedido,
    COUNT(DISTINCT customer_id) AS distinct_customer_count
FROM (
    SELECT 
        ab_test_ref.*,
        CASE 
            WHEN orders.customer_id IS NOT NULL THEN 'true'
            ELSE 'false'
        END AS pedido
    FROM ifood_bronze.ab_test_ref AS ab_test_ref
-- Realiza junção entre tabelas para agregar atributos adicionais
    LEFT JOIN ifood_bronze.orders AS orders
    ON ab_test_ref.customer_id = orders.customer_id
) AS subquery
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 1,2

is_target,pedido,distinct_customer_count
control,true,360542
target,true,445924
target,false,1


Databricks visualization. Run in Databricks to view.

Praticamente todos os usuários do teste A/B, tanto no grupo controle quanto no público abordado, fizeram pedido no período analisado. Portanto, a taxa de conversão não foi um indicador impactante nesse experimento.

### 4.2 Receita por usuário

In [0]:
%sql
select 
  a.is_target, 
  sum(b.order_total_amount) as receita_total, 
  count(distinct b.customer_id) as clientes, 
  sum(b.order_total_amount) / count(distinct b.customer_id) as ticket_medio
from 
  workspace.ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
inner join 
  workspace.default.ab_test_ref a
on 
  a.customer_id = b.customer_id
-- Agrupa os dados para cálculo de métricas agregadas
group by 
  a.is_target

is_target,receita_total,clientes,ticket_medio
control,7.307187305451661E7,360542,202.67229075812696
target,1.0200757006332143E8,445924,228.75550556444915


Databricks visualization. Run in Databricks to view.

O público abordado teve 12,8% a mais de ticket médio do que o grupo controle.

### 4.3 Ticket por pedido

In [0]:
%sql
select 
  a.is_target, 
  sum(b.order_total_amount) as receita_total, 
  count(distinct b.order_id) as pedidos, 
  sum(b.order_total_amount) / count(distinct b.order_id) as ticket_pedido
from 
  workspace.ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
inner join 
  workspace.default.ab_test_ref a
on 
  a.customer_id = b.customer_id
-- Agrupa os dados para cálculo de métricas agregadas
group by 
  a.is_target

is_target,receita_total,pedidos,ticket_pedido
control,7.307187305451661E7,1010738,72.29556329584582
target,1.0200757006332143E8,1416677,72.00481836249295


Databricks visualization. Run in Databricks to view.

O ticket por pedido do grupo controle foi 0,4% maior do que o público abordado. Não foi um fator impactante do teste.

### 4.4 Pedidos por cliente

In [0]:
%sql
select 
  a.is_target, 
  count(distinct b.order_id) as pedidos, 
  count(distinct b.customer_id) as clientes, 
  count(distinct b.order_id) / count(distinct b.customer_id) as pedidos_cliente
from 
  workspace.ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
inner join 
  workspace.default.ab_test_ref a
on 
  a.customer_id = b.customer_id
-- Agrupa os dados para cálculo de métricas agregadas
group by 
  a.is_target

is_target,pedidos,clientes,pedidos_cliente
control,1010738,360542,2.80338490383922
target,1416677,445924,3.176947192795185


Databricks visualization. Run in Databricks to view.

O público abordado da ação fez 13,2% a mais de pedidos por cliente do que o grupo controle.

### 4.5 Resumo dos resultados

A taxa de conversão foi praticamente igual entre os grupos controle e testado, o que indica que esse indicador não foi relevante para avaliar o impacto da campanha.

Entre os usuários abordados pela ação, observou-se:

- +12,8% de ticket médio por cliente em relação ao grupo controle;
- +13,2% de pedidos por cliente, indicando maior engajamento;
- -0,4% no ticket por pedido, variação pequena e sem impacto relevante.

Esses resultados sugerem que a campanha contribuiu para um aumento no valor total movimentado por cliente, impulsionado principalmente pela frequência de pedidos.

### 4.6 Viabilidade Financeira

b) Faça uma análise de viabilidade financeira dessa iniciativa 
como alavanca de crescimento, adotando as premissas que julgar necessárias (explicite as premissas adotadas). 

Premissas adotadas
- Valor do cupom por cliente: R$ 10
- Ticket médio do grupo controle: R$ 202,67
- Ticket médio do grupo testado: R$ 228,75
- Incremento no ticket médio: +R$ 26,08 (+12,8%)
- Margem de contribuição estimada: 30% sobre a receita incremental

Receita incremental por cliente = R$ 26,08  
Lucro incremental (30% de margem) = R$ 7,82  
Custo do cupom = R$ 10,00

ROI = (7,82 - 10,00) / 10,00 = -21,8%

### 4.7 Conclusão

A campanha de cupons com valor de R$ 10 gerou aumento relevante no ticket médio (+12,8%) e na frequência de pedidos (+13,2%) entre os usuários impactados. Apesar de o ROI ainda ser negativo (-21,8%), o aumento no número médio de pedidos indica potencial de retorno no médio prazo, especialmente se o comportamento se mantiver após o incentivo. Com ajustes na segmentação e monitoramento da recorrência, a iniciativa se mostra promissora como alavanca de crescimento.

### 4.8 Propostas
c) Recomende oportunidades de melhoria nessa ação e desenhe uma nova proposta de teste A/B para validar essas hipóteses. 


- Direcionar o cupom para outras segmentações, que tendem a ter maior potencial de ativação, gerando maior retorno financeiro.
- Testar cupons menores (ex: R$ 5 ou R$ 7), avaliando se ainda geram aumento no ticket ou frequência de pedidos com menor custo da campanha.
- Oferecer o cupom apenas para pedidos acima de um valor mínimo (ex: R$ 60), aumentando o ticket médio e diluindo o custo do cupom.
- Avaliar se os clientes impactados mantêm o comportamento mais engajado nas semanas seguintes à campanha, mesmo sem novos incentivos.

| Hipótese                                                                                | Justificativa                                                  |
| --------------------------------------------------------------------------------------- | -------------------------------------------------------------- |
| H1: Segmentações específicas podem gerar maior retorno financeiro                       | A segmentação tende a gerar maior retorno marginal             |
| H2: Cupons de menor valor (R\$ 5 ou R\$ 7) ainda geram aumento significativo de receita | Permite reduzir o custo da ação sem perder efeito              |
| H3: Condicionar o cupom a um valor mínimo de pedido aumenta o ticket médio              | Garante diluição do custo do cupom por pedido                  |
| H4: Novos clientes com cupom se engajam mais no primeiro uso e podem ser retidos        | Estratégia de aquisição com potencial de retorno a longo prazo |


Grupos experimentais

| Grupo   | Descrição da variação                     | Valor do cupom | Condição mínima |
| ------- | ----------------------------------------- | -------------- | --------------- |
| **GC**  | Grupo controle (sem cupom)                | –              | –               |
| **PA1** | Cupom padrão                              | R\$ 10         | Sem mínimo      |
| **PA2** | Cupom reduzido                            | R\$ 5          | Sem mínimo      |
| **PA3** | Cupom com valor mínimo de pedido          | R\$ 10         | Pedido ≥ R\$ 60 |
| **PA4** | Cupom reduzido com valor mínimo de pedido | R\$ 5          | Pedido ≥ R\$ 60 |

**Indicadores a serem avaliados**
- Ticket médio por pedido
- Frequência de pedidos por cliente
- Receita incremental
- Custo por cliente ativado
- Retenção e comportamento nas 2–4 semanas após o fim da campanha

**Hipóteses a validar**
- Cupons com valor mínimo elevam o ticket e reduzem o custo relativo da campanha
- Incentivos menores ainda geram ativação relevante com ROI mais alto
- O comportamento engajado pode persistir mesmo sem novos cupons (efeito prolongado)

## 5. Questão 2

A criação de segmentações permite agrupar usuários de acordo com características e comportamentos similares, possibilitando criar estratégias direcionadas de acordo com o perfil de cada público, facilitando a personalização e incentivando o engajamento, retenção, além de otimização de recursos. Segmentações de usuários são muito utilizadas pelos times de Data, mas a área em que você atua ainda não tem segmentos bem definidos e cada área de Negócio utiliza conceitos diferentes. Por isso, você precisa: 

**a)** Definir as segmentações que fazem sentido especificamente 
para o teste A/B que está analisando. 

Serão exploradas segmentações a partir dessas três variáveis:
- delivery_address_state (estado de entrega do pedido)
- average_ticket (ticket médio dos pedidos do restaurante)
- created_at (data de registro do usuário)

### 5.1 Exploração das variáveis

In [0]:
%sql
/* Ticket médio e frequência de pedidos por região 
De-Para de estados para região para otimizar segmentação */
SELECT CASE
    WHEN delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
    WHEN delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
    WHEN delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
    WHEN delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
    WHEN delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
    ELSE 'Região desconhecida'
END AS delivery_region,
AVG(order_total_amount) AS ticket_medio_pedido,
COUNT(DISTINCT order_id) as pedidos,
COUNT(DISTINCT customer_id) as clientes,
COUNT(DISTINCT order_id)/COUNT(DISTINCT customer_id) as pedidos_cliente
FROM ifood_bronze.orders
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 1

delivery_region,avg_order_total_amount,pedidos,clientes,pedidos_cliente
Nordeste,42.94758552722553,352964,114379,3.085916120966261
Centro-Oeste,43.79082809166243,114304,41687,2.741957924532828
Norte,44.98190579353483,45166,15556,2.9034456158395474
Sudeste,49.928765608513224,1694302,560582,3.022398150493594
Sul,43.18082848205539,226238,81959,2.7603801900950473


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Segmentar a região **sudeste** que está com o maior ticket médio e segunda maior frequência por cliente.

In [0]:
%sql
SELECT
-- Realiza transformação de tipos de dados para garantir consistência nas análises
  CAST(r.price_range AS STRING) AS price_range,
  AVG(o.order_total_amount) AS ticket_medio_pedido,
  COUNT(DISTINCT o.order_id) AS pedidos,
  COUNT(DISTINCT o.customer_id) AS clientes,
  COUNT(DISTINCT o.order_id) / COUNT(DISTINCT o.customer_id) AS pedidos_cliente,
  COUNT(DISTINCT r.id) as restaurantes
FROM
  workspace.ifood_bronze.orders o
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN
  workspace.ifood_bronze.restaurants r
ON
  r.id = o.merchant_id
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY
  1
-- Ordena os resultados com base em colunas de interesse para facilitar a leitura
ORDER BY
  1

price_range,ticket_medio_pedido,pedidos,clientes,pedidos_cliente,restaurantes
1,34.92396255708846,485281,251008,1.9333288182049975,1814
2,37.650704126365774,487518,259901,1.875783471398725,1486
3,48.332815539522024,932550,456310,2.0436764480287524,2524
4,60.36933525628949,384446,215742,1.781971058023009,1018
5,90.76500121452668,143179,80041,1.7888207293761946,449


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Segmentar os restaurantes de price_range 5 pode elevar o ticket médio da ação, o que pode contribuir para que a receita supere o valor do investimento da campanha.

In [0]:
%sql
-- Cria uma CTE (Common Table Expression) para estruturar consultas complexas de forma modular
WITH consumer_data AS (
  SELECT 
    date_format(created_at, 'MM-yyyy') as data_cadastro, 
    customer_id
  FROM 
    workspace.ifood_bronze.consumers
),
order_data AS (
  SELECT 
    o.customer_id,
    AVG(o.order_total_amount) AS ticket_medio_pedido,
    COUNT(DISTINCT o.order_id) AS pedidos,
    COUNT(DISTINCT o.customer_id) AS clientes,
    COUNT(DISTINCT o.order_id) / COUNT(DISTINCT o.customer_id) AS pedidos_cliente,
    COUNT(DISTINCT r.id) as restaurantes
  FROM 
    workspace.ifood_bronze.orders o
-- Realiza junção entre tabelas para agregar atributos adicionais
  LEFT JOIN 
    workspace.ifood_bronze.restaurants r
  ON 
    o.merchant_id = r.id
-- Agrupa os dados para cálculo de métricas agregadas
  GROUP BY ALL
)
SELECT 
  c.data_cadastro,
  AVG(o.ticket_medio_pedido) AS ticket_medio_pedido,
  SUM(o.pedidos) AS pedidos,
  COUNT(DISTINCT c.customer_id) AS clientes,
  AVG(o.pedidos_cliente) AS pedidos_cliente

FROM 
  consumer_data c
-- Realiza junção entre tabelas para agregar atributos adicionais
JOIN 
  order_data o
ON 
  c.customer_id = o.customer_id
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
  1
-- Ordena os resultados com base em colunas de interesse para facilitar a leitura
ORDER BY 
  1

data_cadastro,ticket_medio_pedido,pedidos,clientes,pedidos_cliente
01-2018,49.119833800332174,1463365,404670,3.616193441569674
02-2018,48.11975368533176,139805,56575,2.4711444984533806
03-2018,47.21845114256873,70313,30356,2.31628014231124
04-2018,46.39205138770914,753107,314555,2.3941981529462257


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Os clientes com data de cadastro em janeiro de 2018 possuem maior ticket médio e maior frequência de pedidos. Vale a pena focar nesses consumidores mais antigos.

| Segmento Combinado         | Critérios                                             | Justificativa                                                                                                        |
| -------------------------- | ----------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **Sudeste Gourmet**        | Região Sudeste + Restaurantes de price\_range 5       | Clientes de alto ticket, em restaurantes premium, em região de alto volume — ideal para maximizar receita por pedido |
| **Veteranos Gourmet** | Clientes cadastrados em 01/2018 + price\_range  5 | Clientes antigos com alta frequência e dispostos a gastar mais — podem ter alto engajamento e boa margem             |
| **Food Fiéis Sudeste**  | Região Sudeste + Cadastro em 01/2018                  | Segmento grande, com bom histórico e potencial de alta resposta à campanha                                           |


**b)** Estabelecer quais serão os critérios utilizados para cada segmento sugerido no item a). Utilize os critérios/ferramentas que 
achar necessários, mas lembre-se de explicar o racional utilizado na criação.

**1. Sudeste Gourmet**

Critérios:
- delivery_region = 'Sudeste'
- price_range = 5

Racional:
- A região Sudeste apresentou o maior ticket médio (R$ 49,93) e alta frequência por cliente.
- Restaurantes com price_range 5 têm ticket médio acima de R$ 90, o que dilui melhor o valor do cupom e aumenta o ROI.
- Combinar esses dois filtros identifica clientes com potencial de gasto elevado e boa recorrência, ideal para ações de conversão de alto valor.

**2. Veteranos Gourmet**

Critérios:
- data_cadastro = 01/2018
- price_range = 5

Racional:
- Esse segmento reúne clientes antigos, com longa jornada no app e que seguem ativos, realizando pedidos em restaurantes premium.
- Frequência acima da média: clientes cadastrados em janeiro de 2018 fazem em média 3,6 pedidos por cliente.
- Restaurantes com price_range 5 possuem ticket médio de R$ 90,77.
- Clientes de janeiro de 2018 têm ticket médio de R$ 49,12.

**3. Food Fiéis Sudeste**

Critérios:
- data_cadastro = 01/2018
- delivery_region = 'Sudeste'

Racional:
- Esse grupo representa usuários experientes e leais, concentrados na região de maior volume e valor da operação do iFood.
- Alta frequência de pedidos com 3,6 pedidos por cliente (cadastro 01/2018).
- Maior ticket médio regional: sudeste com R$ 49,93, superando todas as outras regiões.
- Grande volume de pedidos: Sudeste teve 1.694.302 pedidos no período analisado, sendo o principal mercado.

### 5.2 Análise do teste A/B com novos segmentos

**c)** Analisar os resultados do teste A/B com base nos segmentos definidos nos itens a) e b). 

#### 5.21 Testando segmentação Sudeste Gourmet

In [0]:
%sql
SELECT 
    a.is_target, 
    SUM(b.order_total_amount) AS receita_total, 
    COUNT(DISTINCT b.customer_id) AS clientes, 
    SUM(b.order_total_amount) / COUNT(DISTINCT b.customer_id) AS ticket_medio,
    -- Case when formulado para transformar estado em região
    CASE
        WHEN o.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
        WHEN o.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
        WHEN o.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
        WHEN o.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
        WHEN o.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
        ELSE 'Região desconhecida'
    END AS delivery_region
FROM 
    ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
    ifood_bronze.ab_test_ref a ON a.customer_id = b.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
    ifood_bronze.orders o ON a.customer_id = o.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
    ifood_bronze.restaurants r ON o.merchant_id = r.id
-- Aplica filtros nos dados para considerar apenas registros relevantes ao experimento
WHERE 
    CASE
        WHEN o.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
        WHEN o.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
        WHEN o.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
        WHEN o.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
        WHEN o.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
        ELSE 'Região desconhecida'
    END = 'Sudeste'
    AND r.price_range = 5 -- Price Range dos restaurantes gourmet
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
    a.is_target, 
    CASE
        WHEN o.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
        WHEN o.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
        WHEN o.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
        WHEN o.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
        WHEN o.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
        ELSE 'Região desconhecida'
    END

is_target,receita_total,clientes,ticket_medio,delivery_region
target,1.1246750045726156E8,37408,3006.509315046556,Sudeste
control,7.52825694929831E7,27471,2740.4378978917075,Sudeste


Databricks visualization. Run in Databricks to view.

O ticket médio do PA é 9,7% maior do que do GC. Porém, em relação à segmentação geral, esse é um resultado de ticket 13x maior para o público abordado, o que contribui para aumentar a receita da ação.

In [0]:
%sql
SELECT 
  a.is_target, 
  COUNT(DISTINCT b.order_id) AS pedidos, 
  COUNT(DISTINCT b.customer_id) AS clientes, 
  COUNT(DISTINCT b.order_id) / COUNT(DISTINCT b.customer_id) AS pedidos_cliente,
  CASE
      WHEN b.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
      WHEN b.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
      WHEN b.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
      WHEN b.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
      WHEN b.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
      ELSE 'Região desconhecida'
  END AS delivery_region
FROM 
  workspace.ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.default.ab_test_ref a
    ON a.customer_id = b.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN
  workspace.ifood_bronze.restaurants r
    ON b.merchant_id = r.id
-- Aplica filtros nos dados para considerar apenas registros relevantes ao experimento
WHERE    
  b.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES')
  AND r.price_range = 5 -- Price Range dos restaurantes gourmet
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
  a.is_target,
  CASE
      WHEN b.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
      WHEN b.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
      WHEN b.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
      WHEN b.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
      WHEN b.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
      ELSE 'Região desconhecida'
  END

is_target,pedidos,clientes,pedidos_cliente,delivery_region
target,70583,37408,1.8868423866552608,Sudeste
control,49472,27471,1.8008809289796512,Sudeste


Databricks visualization. Run in Databricks to view.

A frequência de pedidos por cliente é 4,4% maior no público abordado.

#### 5.22 Testando segmentação Veteranos Gourmet

In [0]:
%sql
SELECT 
  a.is_target, 
  date_format(c.created_at, 'MM-yyyy') as data_cadastro,
  SUM(o.order_total_amount) AS receita_total, 
  COUNT(DISTINCT o.customer_id) AS clientes, 
  SUM(o.order_total_amount) / COUNT(DISTINCT o.customer_id) AS ticket_medio
FROM 
  workspace.ifood_bronze.orders o
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.default.ab_test_ref a
    ON a.customer_id = o.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.ifood_bronze.restaurants r
    ON o.merchant_id = r.id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN
  workspace.ifood_bronze.consumers c
    ON o.customer_id = c.customer_id
-- Aplica filtros nos dados para considerar apenas registros relevantes ao experimento
WHERE 
  r.price_range = 5
  AND date_format(c.created_at, 'MM-yyyy') = '01-2018' -- Mês de cadastro dos usuários veteranos
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
  a.is_target,
  date_format(c.created_at, 'MM-yyyy')

is_target,data_cadastro,receita_total,clientes,ticket_medio
control,01-2018,5099733.669639587,20599,247.57190492934546
target,01-2018,7321007.4222512245,28813,254.08695457783725


Databricks visualization. Run in Databricks to view.

O ticket médio do PA foi 2,6% maior do que do GC.

In [0]:
%sql
SELECT 
  a.is_target, 
  count(distinct o.order_id) as pedidos, 
  count(distinct o.customer_id) as clientes, 
  count(distinct o.order_id) / count(distinct o.customer_id) as pedidos_cliente
FROM 
  workspace.ifood_bronze.orders o
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.default.ab_test_ref a
    ON a.customer_id = o.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.ifood_bronze.restaurants r
    ON o.merchant_id = r.id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN
  workspace.ifood_bronze.consumers c
    ON o.customer_id = c.customer_id
-- Aplica filtros nos dados para considerar apenas registros relevantes ao experimento
WHERE 
  r.price_range = 5
  AND date_format(c.created_at, 'MM-yyyy') = '01-2018' -- Mês de cadastro dos usuários veteranos
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
  a.is_target,
  date_format(c.created_at, 'MM-yyyy')

is_target,pedidos,clientes,pedidos_cliente
control,38096,20599,1.8494101655420165
target,54954,28813,1.9072640821851248


Databricks visualization. Run in Databricks to view.

A frequência por cliente do PA foi 2,7% superior que do GC.

#### 5.23 Testando segmentação Food Fiéis Sudeste

In [0]:
%sql
SELECT 
  a.is_target, 
  SUM(b.order_total_amount) AS receita_total, 
  COUNT(DISTINCT b.customer_id) AS clientes, 
  SUM(b.order_total_amount) / COUNT(DISTINCT b.customer_id) AS ticket_medio
FROM 
  workspace.ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.default.ab_test_ref a
    ON a.customer_id = b.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.ifood_bronze.consumers c
    ON c.customer_id = b.customer_id
-- Aplica filtros nos dados para considerar apenas registros relevantes ao experimento
WHERE
  date_format(c.created_at, 'MM-yyyy') = '01-2018'
  AND
    CASE
      WHEN b.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
      WHEN b.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
      WHEN b.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
      WHEN b.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
      WHEN b.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
      ELSE 'Região desconhecida'
    END = 'Sudeste'
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
  a.is_target

is_target,receita_total,clientes,ticket_medio
control,3.2139376055247575E7,123320,260.6177104707069
target,4.618653559191705E7,158721,290.9919644654271


Databricks visualization. Run in Databricks to view.

O ticket médio do PA é 11,6% maior do que do GC.

In [0]:
%sql
SELECT 
  a.is_target, 
  count(distinct b.order_id) as pedidos, 
  count(distinct b.customer_id) as clientes, 
  count(distinct b.order_id) / count(distinct b.customer_id) as pedidos_cliente
FROM 
  workspace.ifood_bronze.orders b
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.default.ab_test_ref a
    ON a.customer_id = b.customer_id
-- Realiza junção entre tabelas para agregar atributos adicionais 
INNER JOIN 
  workspace.ifood_bronze.consumers c
    ON c.customer_id = b.customer_id
-- Aplica filtros nos dados para considerar apenas registros relevantes ao experimento
WHERE
  date_format(c.created_at, 'MM-yyyy') = '01-2018'
  AND
    CASE
      WHEN b.delivery_address_state IN ('SP', 'RJ', 'MG', 'ES') THEN 'Sudeste'
      WHEN b.delivery_address_state IN ('RS', 'SC', 'PR') THEN 'Sul'
      WHEN b.delivery_address_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
      WHEN b.delivery_address_state IN ('BA', 'PE', 'CE', 'AL', 'RN', 'PB', 'SE', 'PI', 'MA') THEN 'Nordeste'
      WHEN b.delivery_address_state IN ('AM', 'PA', 'AC') THEN 'Norte'
      ELSE 'Região desconhecida'
    END = 'Sudeste'
-- Agrupa os dados para cálculo de métricas agregadas
GROUP BY 
  a.is_target

is_target,pedidos,clientes,pedidos_cliente
control,414705,123320,3.362836522867337
target,598049,158721,3.767926109336509


Databricks visualization. Run in Databricks to view.

A frequência de pedidos por cliente do PA foi 11,9% maior do que do GC.

### 5.3 Resumo dos novos segmentos

| Segmento               | Diferença no Ticket Médio | Significância (Ticket) | Diferença na Frequência de Pedidos | Significância (Frequência) |
| ---------------------- | ------------------------- | ---------------------- | ---------------------------------- | -------------------------- |
| **Sudeste Gourmet**    | **+9,7%** (R\$ 266,07)    | ✅ Alta                 | **+4,4%** (+0,0859)                | ✅ Significativa            |
| **Veteranos Gourmet**  | **+2,6%** (R\$ 6,51)      | ⚠️ Limítrofe           | **+2,7%** (+0,0579)                | ⚠️ Baixa            |
| **Food Fiéis Sudeste** | **+11,6%** (R\$ 30,38)    | ✅ Alta                 | **+11,9%** (+0,4051)               | ✅ Significativa     |


## 6. Questão 3
Com base na análise que realizou nas questões 1 e 2, sugira os próximos passos que o iFood deve tomar. Lembre-se que você precisa defender suas sugestões para as lideranças de Negócio, por isso não esqueça de incluir uma previsão de impacto (financeiro ou não) caso o iFood siga com a sua recomendação. Fique à vontade para sugerir melhorias no processo/teste e para propor diferentes estratégias de acordo com cada segmento de usuário. 

**1 - Escalar a campanha para o segmento “Sudeste Gourmet”**

- Motivo: Maior ticket médio absoluto entre os grupos (R$ 3.006,51), 9,7% superior ao controle, e frequência também maior (+4,4%), ambos com alta significância estatística.
- Impacto previsto: A cada 100 mil clientes atingidos, a ação pode gerar até R$ 26,6 milhões em receita incremental bruta, com investimento de apenas R$ 1 milhão em cupons (R$ 10 por cliente).
- Ação sugerida: Expandir nacionalmente para clientes com perfil semelhante (alta propensão de gasto e frequência elevada), testando diferentes valores de cupom para otimização de custo-benefício.

**2 - Reforçar campanhas com o grupo “Food Fiéis Sudeste”**

- Motivo: Ticket médio 11,6% superior ao GC e frequência 11,9% maior, com a maior base de clientes entre os grupos analisados (158 mil).
- Impacto previsto: Alto volume de receita incremental ao longo do tempo, se trabalhado com estratégia de retenção contínua (ex: cupons sazonais, gamificação ou cashback).
- Ação sugerida: Criar jornadas de CRM mais longas para esse público, combinando incentivo com reconhecimento (ex: selo de fidelidade, pushs personalizados, badges no app).

**3 - Aprofundar o teste no grupo “Veteranos Gourmet”**
- Motivo: Ticket médio e frequência superiores ao controle, mas com impacto marginal.
- Ação sugerida: Aplicar testes A/B com variações de abordagem, como:
- Comunicação emocional voltada à fidelidade ("Você está com a gente desde o começo")
- Incentivos não-financeiros (ex: prioridade em lançamento de funcionalidades, entrega expressa)
- Objetivo: Identificar se o comportamento desse grupo é mais sensível a outros tipos de estímulo que não apenas cupons.

**4 -  Melhorias nos processos de teste e segmentação**
- Padronizar critérios de segmentação entre áreas, garantindo que análises como esta possam ser replicadas e comparadas ao longo do tempo.
- Criar um repositório de segmentos validados (ex: cluster “Sudeste Gourmet”) para reuso em campanhas e automação.
- Implementar modelos de propensão a engajamento, combinando variáveis como região, histórico de pedidos e perfil de restaurantes consumidos.
- Automatizar os testes A/B com pipelines Delta + alertas, para acelerar aprendizado e escalar resultados com agilidade.



## 7. Considerações finais

Com base nas análises realizadas, recomenda-se escalar a campanha de cupons para segmentos com maior retorno validado, como o Sudeste Gourmet, que apresentou ticket médio 9,7% maior e ROI estimado em 99x, e o grupo Food Fiéis Sudeste, com aumentos significativos em ticket e frequência. A segmentação permitiu identificar públicos com alto potencial de receita e engajamento, reforçando a importância de estratégias personalizadas. Sugere-se ainda testar variações de incentivo (ex: cupons menores ou cashback) e ampliar ações para retenção de clientes veteranos. Por fim, padronizar os critérios de segmentação entre áreas e automatizar testes A/B com pipelines pode acelerar aprendizados e maximizar resultados futuros.